In [51]:
import pandas as pd
import os

# Set Working Directory
os.chdir('C:\\Users\\user\\Desktop\\Summer 2020\\Research Assistant\\Sandra\\Work\\Data')
os.getcwd()

'C:\\Users\\user\\Desktop\\Summer 2020\\Research Assistant\\Sandra\\Work\\Data'

define keywords relevant to british, climate, pro and anti brexit

In [52]:
british_keywords = ["brexit", "european union", " eu ", "british", "british identity", "british passport", 
                    "british culture", "british heritage", "british goods", "british products","british manufacturing", 
                    "made in britain"]
climate_keywords = ["climate change", "paris agreement", "environment", "global warming", "unfccc"]
pro = ['leave', 'vote leave', 'eu out', 'take control']
anti = ['uk in eu', 'remain', 'eu in', 'vote in']

In [53]:
headlines_df = pd.read_csv('express.csv', index_col = 'date')

In [54]:
headlines_df

,headline,newspaper
date,,
1-3-2015,Prince Andrew 'frozen out by Charles over dama...,Express
1-3-2015,EXCLUSIVE: Migrants to put Britain's populatio...,Express
1-3-2015,EXCLUSIVE: Jihadi John exposed by web error: K...,Express
1-3-2015,Missing Becky Watts: Two arrested as family ad...,Express
1-3-2015,We can all benefit from a positive approach to...,Express
1-3-2015,Footage emerges appearing to show missing Lond...,Express
1-3-2015,Spring fresh: The best tools to clean your home,Express
1-3-2015,'Maybe it will come my way' Kylie Minogue admi...,Express
1-3-2015,"Amara, Rimmel, Microsoft Lumia: The things we ...",Express


In [55]:
headlines_df['classification'] = ''
classification_lst = []

def headline_classifier(string, keyword_lst_class_1, keyword_lst_class_2, name_class_1 = 'class_1', name_class_2 = 'class_2'):
    '''Classifies a list of strings according to keyword lists for class_1 and class_2'''
    string = string.lower()
    count_1 = 0
    count_2 = 0
    for keyword in keyword_lst_class_1:
        if keyword in string:
            count_1 =+ 1
    for keyword in keyword_lst_class_2:

        if keyword in string:
            count_2 =+ 1
    
    if count_1 > 0 and count_2 > 0:
        return 'both'
    elif count_1 > 0:
        return name_class_1
    elif count_2 > 0:
        return name_class_2
    else:
        return 'none'

In [56]:
def headline_classification(data, keyword_lst_class_1, keyword_lst_class_2, var_name='classification', name_class_1 = 'class_1', name_class_2 = 'class_2'):
    
    '''For every headline in the data, assign value based on name_class_1 and name_class_2 to var_name using keyword_lst_class_1 and keyword_lst_class_2'''
    n_row, n_col = data.shape
    data_lst = list(data['headline'].values)

    #Headline classification
    classification_lst = []
    for i in range(n_row):
        data_i = str(data_lst[i]) #Include this becasue of potential nan values
        classification_lst.append(headline_classifier(data_i, keyword_lst_class_1, keyword_lst_class_2, name_class_1, name_class_2))
    data[var_name] = classification_lst
    

In [57]:
col1_name = 'classification'
headline_classification(headlines_df, british_keywords, climate_keywords, var_name=col1_name, name_class_1='Brexit', name_class_2='Climate')

In [58]:
headlines_df.classification.value_counts()

none       207415
Brexit      11789
Climate       207
both           11
Name: classification, dtype: int64

In [59]:
headlines_df

,headline,newspaper,classification
date,,,
1-3-2015,Prince Andrew 'frozen out by Charles over dama...,Express,none
1-3-2015,EXCLUSIVE: Migrants to put Britain's populatio...,Express,none
1-3-2015,EXCLUSIVE: Jihadi John exposed by web error: K...,Express,none
1-3-2015,Missing Becky Watts: Two arrested as family ad...,Express,none
1-3-2015,We can all benefit from a positive approach to...,Express,none
1-3-2015,Footage emerges appearing to show missing Lond...,Express,none
1-3-2015,Spring fresh: The best tools to clean your home,Express,none
1-3-2015,'Maybe it will come my way' Kylie Minogue admi...,Express,none
1-3-2015,"Amara, Rimmel, Microsoft Lumia: The things we ...",Express,none


Limit dataset to articles that are focused towards Brexit.

Note: This is dependent on how we define British Keywords (see above)

In [60]:
brexit_df = headlines_df[headlines_df[col1_name]=="Brexit"]

In [61]:
brexit_df

,headline,newspaper,classification
date,,,
1-3-2015,'We must monitor Islamic student societies wit...,Express,Brexit
2-3-2015,Now EU targets halogen bulbs: Brussels could B...,Express,Brexit
2-3-2015,In bed with Brook: British model Kelly shows o...,Express,Brexit
2-3-2015,Tory MP says Britain would be better off witho...,Express,Brexit
2-3-2015,Two out of three British laws were made in Bru...,Express,Brexit
2-3-2015,Fresh search for missing Ben Needham with Brit...,Express,Brexit
2-3-2015,Guinness team up with Piebury Corner to celebr...,Express,Brexit
3-3-2015,Jihadi fighters are portrayed as 'eye candy' t...,Express,Brexit
3-3-2015,'Broadchurch is grim and boring' Celia Imrie s...,Express,Brexit


classifying brexit related headlines into pro and anti brexit

In [62]:
col2_name = 'brexit'
headline_classification(brexit_df, pro, anti, var_name=col2_name, name_class_1='pro', name_class_2='anti')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [63]:
brexit_df.brexit.value_counts()

none    10917
pro       431
anti      408
both       33
Name: brexit, dtype: int64

In [64]:
brexit_df

,headline,newspaper,classification,brexit
date,,,,
1-3-2015,'We must monitor Islamic student societies wit...,Express,Brexit,none
2-3-2015,Now EU targets halogen bulbs: Brussels could B...,Express,Brexit,none
2-3-2015,In bed with Brook: British model Kelly shows o...,Express,Brexit,none
2-3-2015,Tory MP says Britain would be better off witho...,Express,Brexit,none
2-3-2015,Two out of three British laws were made in Bru...,Express,Brexit,none
2-3-2015,Fresh search for missing Ben Needham with Brit...,Express,Brexit,none
2-3-2015,Guinness team up with Piebury Corner to celebr...,Express,Brexit,none
3-3-2015,Jihadi fighters are portrayed as 'eye candy' t...,Express,Brexit,none
3-3-2015,'Broadchurch is grim and boring' Celia Imrie s...,Express,Brexit,none


In [ ]:
brexit_df.to_csv(r'C:\\Users\\user\\Desktop\\Summer 2020\\Research Assistant\\Sandra\\Work\\Data\\brexit.csv')